In [3]:
import openai, os, json

In [4]:
openai.api_key = os.environ.get('OPENAI_API_KEY')

In [5]:
def query(prompt, **kwargs):
    """
    wrapper for the API to save the prompt and the result
    """
    args = {
    "engine":"text-davinci-003",
    "temperature":0,
    "max_tokens":400,
    "stop":"END",
    }

    for key in kwargs:
        args[key] = kwargs[key]

    r = openai.Completion.create(prompt=prompt, **args)
    return r["choices"][0]["text"].strip()


In [6]:
class StoryWriter:

  def __init__(self, theme, genre, style):
    self.theme = theme
    self.genre = genre
    self.style = style
    self.queue = []
    self.story = []

  def createStoryOverview(self):
    """
    Creates a story outline based on the theme, genre, and style
    """
    overviewForStoryPrompt = """ /* Generate an outline for a story

    type character{{
      "name": str,
      "profession": str,
      "age": int,
    }}

    interface outline {{
      "setting": str, //where the section takes place
      "characters": character[], //the character names
      "conflict": str, //what conflict needs to get solved
      "plotTwist": str, //any plot twists to occur
      "resolution": str // how the story concludes
    }}

    const generatePlot = (theme, genre, style) : outline =>{{
        return ai.generatePlot(theme)
    }}


    theme = '{}'
    genre = '{}'
    style = '{}'
    //make sure to include the keys in quotes so we can json.parse
    console.log("Out: " + generatePlot(theme, genre, style))
    Out:"""
    prompt = overviewForStoryPrompt.format(self.theme, self.genre, self.style)
    self.overview = query(prompt)
    print('overview: ', self.overview)

  def createStoryOutline(self):
    """
    Creates a story outline based on the theme, genre, and style
    """
    outlineForStoryPrompt = """ /* Generate an outline for a story

    type section{{
      "name": str, // the name of the section
      "description": str, // a brief overview of what happens in the section
    }}

    interface outline {{
      "parts": section[] //all of the sections in the story
    }}

    const generateOutline = (storyOverview) : outline =>{{
        return ai.generateOutline(storyOverview)
    }}

    overview = '{}'

    //make sure to include the keys in quotes so we can json.parse
    console.log("Out: " + generateOutline(overview))
    Out:"""

    prompt = outlineForStoryPrompt.format(self.overview)
    self.outline = query(prompt)
    print('self.outline: ', self.outline)

  def expandOutline(self):
    """
    Expand an existing outline to include sub-sections
    """
    expandOutlinePrompt = """ /* Generate the sub-outline for each

    type section{{
      "name": str, // the name of the section
      "description": str, // a brief overview of what happens in the section
    }}

    interface outline {{
      "parts": section[] //all of the sections in the story
    }}

    const generateOutline = (outline) : outline =>{{
        return ai.buildDeeperOutline(outline)
    }}

    overview = '{}'

    //make sure to include the keys in quotes so we can json.parse
    console.log("Out: " + generateOutline(overview))
    Out:"""

    prompt = outlineForStoryPrompt.format(self.overview)
    self.outline = query(prompt)
    print('self.outline: ', self.outline)



In [8]:
storyBuilder = StoryWriter('space', 'sci-fi', 'action')
storyBuilder.createStoryOverview()
storyBuilder.createStoryOutline()

overview:  {
      "setting": "Space Station Alpha",
      "characters": [
        {
          "name": "John Smith",
          "profession": "Space Pilot",
          "age": 32
        },
        {
          "name": "Jane Doe",
          "profession": "Space Engineer",
          "age": 28
        }
      ],
      "conflict": "John and Jane must work together to save the Space Station Alpha from a mysterious alien threat.",
      "plotTwist": "John and Jane discover that the alien threat is actually a group of humans from a parallel universe.",
      "resolution": "John and Jane are able to use their combined knowledge and skills to save the Space Station Alpha and the parallel universe from destruction."
    }
self.outline:  {
      "parts": [
        {
          "name": "Introduction",
          "description": "Introduce the setting, characters, and conflict of the story."
        },
        {
          "name": "Rising Action",
          "description": "John and Jane must work together

In [10]:
print(storyBuilder.outline)

{
      "parts": [
        {
          "name": "Introduction",
          "description": "Introduce the setting, characters, and conflict of the story."
        },
        {
          "name": "Rising Action",
          "description": "John and Jane must work together to save the Space Station Alpha from the alien threat."
        },
        {
          "name": "Plot Twist",
          "description": "John and Jane discover that the alien threat is actually a group of humans from a parallel universe."
        },
        {
          "name": "Climax",
          "description": "John and Jane must use their combined knowledge and skills to save the Space Station Alpha and the parallel universe from destruction."
        },
        {
          "name": "Resolution",
          "description": "John and Jane are successful in their mission and the Space Station Alpha and the parallel universe are saved."
        }
      ]
    }


In [15]:
queue = json.loads(storyBuilder.outline)['parts']

In [17]:
currentNode = queue.pop(0)

In [18]:
currentNode

{'name': 'Introduction',
 'description': 'Introduce the setting, characters, and conflict of the story.'}

In [19]:
story = []
queueHead = queue[:2]

[{'name': 'Rising Action',
  'description': 'John and Jane must work together to save the Space Station Alpha from the alien threat.'},
 {'name': 'Plot Twist',
  'description': 'John and Jane discover that the alien threat is actually a group of humans from a parallel universe.'}]